In [1]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
from test_utils import test
from helpers import predict_labels
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will use a toy dataset from sklearn.

In [2]:
from sklearn import datasets

#Load dataset
sklearn_dataset = datasets.load_breast_cancer()
Xx  = sklearn_dataset.data
y = sklearn_dataset.target * 2 - 1    # labels must be in {-1, 1} for the hinge loss
X = np.ones((Xx.shape[0], Xx.shape[1] + 1 ))   
X[:, :-1] = Xx
print("(N, D) =", X.shape)

(N, D) = (569, 31)


## Prepare cost and prediction functions

In [3]:
def hinge_loss(z):
    return max(0, 1-z)


def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective, equation (1) in the exercise pdf),
        that is loss plus regularizer.
        
    Args:
        X: the full dataset matrix, shape = (num_examples, num_features)
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        w: shape = (num_features)
    
    Returns:
        scalar, non-negative
        
    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1, 2, 3], [4, 5, 6]])
    >>> w_test = np.array([0, 1, 5])
    >>> calculate_primal_objective(y_test, x_test, w_test, 1)
    49.0
    """
    return sum([hinge_loss(y[n] * np.dot(X[n], w)) for n in range(len(X))]) + (lambda_ / 2) * np.dot(w, w)

In [4]:
test(calculate_primal_objective)

✅ Your `calculate_primal_objective` passed 4 tests.


In [5]:
def calculate_accuracy(y, X, w):
    """compute the accuracy on the given dataset (X, y) using the model w.
    
    Args:
        X: the full dataset matrix, shape = (num_examples, num_features)
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        w: shape = (num_features)
    
    Returns:
        scalar, between 0 and 1
        
    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1, 2, 3], [4, 5, 6]])
    >>> w_test = np.array([0, 1, 5])
    >>> calculate_accuracy(y_test, x_test, w_test)
    0.5
    """
    y_pred = predict_labels(weights=w, data=X)
    n_errors = (y != y_pred).sum()
    acc = 1 - (n_errors / len(y))
    return acc

In [6]:
test(calculate_accuracy)

✅ Your `calculate_accuracy` passed 4 tests.


## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [7]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    
    Args:
        X: the dataset matrix, shape = (num_examples, num_features)
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        w: shape = (num_features)
        lambda_: positive scalar number
        n: the index of the (one) datapoint we have sampled
        num_examples: N
    
    Returns:
        numpy array, shape = (num_features)
    
    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1, 2, 3], [4, 5, 6]])
    >>> w_test = np.array([0, 1, 5])
    >>> calculate_stochastic_gradient(y_test, x_test, w_test, 1, 1, 2)
    array([ 8, 11, 17])
    """
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!

    x_n, y_n = X[n], y[n]
    z = y_n * np.dot(x_n, w)
    sg = lambda_ * w
    if z < 1:
        sg += - y_n * x_n * num_examples
    return sg


In [8]:
test(calculate_stochastic_gradient)

✅ Your `calculate_stochastic_gradient` passed 4 tests.


Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [9]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 2 * int(1e5)
    gamma = 1e-4
    lambda_ = int(1e4)   # big because scales with N due to the formulation of the problem (not an averaged loss)

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma / (it+1) * grad   
        
        if it % 20000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
            
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))


sgd_for_svm_demo(y, X)

iteration=0, cost=33201291.068423286
iteration=20000, cost=191.0219167901912
iteration=40000, cost=173.5753288344948
iteration=60000, cost=277.74640196013416
iteration=80000, cost=159.2257199515269
iteration=100000, cost=128.96679851096096
iteration=120000, cost=167.019442266154
iteration=140000, cost=128.44207289363098
iteration=160000, cost=130.13709937123005
iteration=180000, cost=125.00121404355424
training accuracy = 0.9226713532513181


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [46]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    
    Args:
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        X: the dataset matrix, shape = (num_examples, num_features)
        lambda_: positive scalar number
        alpha: vector of dual coordinates, shape = (num_examples)
        w: vector of primal parameters, shape = (num_features)
        n: the coordinate to be updated
        
    Returns:
        w: updated vector of primal parameters, shape = (num_features)
        alpha: updated vector of dual parameters, shape = (num_examples)
        
    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1., 2., 3.], [4., 5., 6.]])
    >>> w_test = np.array([-0.3, -0.3, -0.3])
    >>> alpha_test = np.array([.1, .1])
    >>> calculate_coordinate_update(y_test, x_test, 1, alpha_test, w_test, 0)
    (array([-0.1,  0.1,  0.3]), array([0.3, 0.1]))
    """        
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    gamma_star = (lambda_ / (np.dot(x_n, x_n))) * (1 - y_n * np.dot(w, x_n))
    if old_alpha_n + gamma_star > 1:
        gamma_star = 1 - old_alpha_n
    if old_alpha_n + gamma_star < 0:
        gamma_star = - old_alpha_n
    
    alpha_n = old_alpha_n + gamma_star
    alpha[n] = alpha_n
    w += (gamma_star / lambda_) * y_n * x_n
    return w, alpha

In [47]:
test(calculate_coordinate_update)

✅ Your `calculate_coordinate_update` passed 5 tests.


In [48]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem.
    
    Args:
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        X: the dataset matrix, shape = (num_examples, num_features)
        w: vector of primal parameters, shape = (num_features)
        alpha: vector of dual coordinates, shape = (num_examples)
        lambda_: non negative scalar number

    Output:
        scalar

    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1., 2., 3.], [4., 5., 6.]])
    >>> w_test = np.array([-0.3, -0.3, -0.3])
    >>> alpha_test = np.array([.1, .1])
    >>> calculate_dual_objective(y_test, x_test, w_test, alpha_test, 1)
    0.065
    """
    Y = np.diag(y)
    return np.round(np.sum(alpha) - (1 / (2 * lambda_)) * alpha @ Y @ X @ X.T @ Y @ alpha, 8)

In [49]:
test(calculate_dual_objective)

✅ Your `calculate_dual_objective` passed 5 tests.


In [51]:
# Notice that the gap is going to 0
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 2*int(1e5)
    lambda_ = int(1e4)

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 20000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

iteration=0, primal:532.29760, dual:0.00113, gap:532.29647
iteration=20000, primal:140.28835, dual:55.77746, gap:84.51088
iteration=40000, primal:130.45849, dual:87.79557, gap:42.66292
iteration=60000, primal:125.70315, dual:103.98103, gap:21.72212
iteration=80000, primal:128.36930, dual:111.50853, gap:16.86077
iteration=100000, primal:125.71272, dual:115.31696, gap:10.39576
iteration=120000, primal:123.59170, dual:117.73722, gap:5.85448
iteration=140000, primal:140.47690, dual:119.35277, gap:21.12413
iteration=160000, primal:122.86241, dual:120.27787, gap:2.58454
iteration=180000, primal:123.60539, dual:120.84582, gap:2.75957
training accuracy = 0.9244288224956063
